In [21]:
import os
import pandas as pd

## SemEval 2017 Dataframe

In [22]:
def map_label(label):
    if label == "negative":
        return 0
    elif label == "neutral":
        return 1
    elif label == "positive":
        return 2
    else:
        return None

In [23]:
def process_file(file_path):
    tweets = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split('\t')
            if len(parts) == 3:
                tweet_id, label, tweet = parts
                mapped_label = map_label(label)
                if mapped_label is not None:
                    tweets.append(tweet)
                    labels.append(mapped_label)
    return pd.DataFrame({'Tweet': tweets, 'label': labels})

In [24]:
input_dir = 'twitter_data/custom_data/Semeval_2017/'

semeval2017_data = pd.DataFrame()

In [25]:
for file_name in os.listdir(input_dir):
    if file_name.endswith('.txt'):
        file_path = os.path.join(input_dir, file_name)
        file_data = process_file(file_path)
        semeval2017_data = pd.concat([semeval2017_data, file_data], ignore_index=True)

## SemEval 2018 Dataframe

In [40]:
input_dir = 'twitter_data/custom_data/Semeval_2018/'

In [41]:
semeval2018_data = pd.DataFrame()

In [46]:
def map_emotion_to_sentiment(row):
    positive_emotions = ['joy', 'love', 'optimism', 'trust', 'anticipation']
    negative_emotions = ['anger', 'disgust', 'fear', 'pessimism', 'sadness']
    neutral_emotions = ['surprise']

    positive = any(row[emotion] == 1 for emotion in positive_emotions)
    negative = any(row[emotion] == 1 for emotion in negative_emotions)
    neutral = any(row[emotion] == 1 for emotion in neutral_emotions)

    if positive and negative:
        return 1
    elif positive:
        return 2
    elif negative:
        return 0
    elif neutral:
        return 1
    else:
        return 1

In [47]:
for file_name in os.listdir(input_dir):
    if file_name.endswith('.txt'):
        file_path = os.path.join(input_dir, file_name)

        file_data = pd.read_csv(file_path, sep='\t')

        semeval2018_data = pd.concat([semeval2018_data, file_data], ignore_index=True)

In [48]:
semeval2018_data['sentiment'] = semeval2018_data.apply(map_emotion_to_sentiment, axis=1)

In [51]:
len(semeval2018_data[semeval2018_data['sentiment'] == 0])

7228

## SenWave Dataset

In [ ]:
# Import SenWave Dataset
# Convert the labels to positive, neutral, or negative
# Create new tweets 